In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import argparse
import yaml
import time
import datetime
import cv2
import numpy as np
import pandas as pd
import random

from skimage import io, transform
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
from scipy import ndimage, misc

import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms, utils
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
import torchvision
import torch
from torch.autograd import Variable
import torch.optim as optim
from sklearn.metrics import mean_absolute_error

from google.colab import drive
import os
import warnings
from collections import defaultdict
from sklearn.model_selection import KFold
import time
from scipy import stats
import sys


In [ ]:
categorical_columns = ['TRAPTYPE', 'ATTRACTANTSUSED', 'SETTIMEOFDAY', 'COLLECTTIMEOFDAY', 'GENUS', 'SPECIES', 'TRAPID','TRAPSITE']
numerical_columns = ['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR','TRAPCOLLECT', 'DIFF_DAYS']

weather_columns = ['sunriseTime', 'sunsetTime', 'moonPhase', 
                     'precipIntensity', 'precipIntensityMax', 'precipProbability', 'temperatureHigh', 'temperatureHighTime',
                     'temperatureLow', 'temperatureLowTime', 'apparentTemperatureHigh', 'apparentTemperatureHighTime',
                     'apparentTemperatureLow', 'apparentTemperatureLowTime', 'dewPoint', 'humidity', 'pressure', 'windSpeed', 'windGust', 
                     'windGustTime', 'windBearing', 'cloudCover', 'uvIndex', 'uvIndexTime','visibility', 'temperatureMin',
                     'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime', 'apparentTemperatureMin','apparentTemperatureMinTime', 'apparentTemperatureMax',
                     'apparentTemperatureMaxTime','time']

for i in range(1,15):
    for item in weather_columns:
        numerical_columns.append(item + str(i))

print(numerical_columns)

['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR', 'TRAPCOLLECT', 'DIFF_DAYS', 'sunriseTime1', 'sunsetTime1', 'moonPhase1', 'precipIntensity1', 'precipIntensityMax1', 'precipProbability1', 'temperatureHigh1', 'temperatureHighTime1', 'temperatureLow1', 'temperatureLowTime1', 'apparentTemperatureHigh1', 'apparentTemperatureHighTime1', 'apparentTemperatureLow1', 'apparentTemperatureLowTime1', 'dewPoint1', 'humidity1', 'pressure1', 'windSpeed1', 'windGust1', 'windGustTime1', 'windBearing1', 'cloudCover1', 'uvIndex1', 'uvIndexTime1', 'visibility1', 'temperatureMin1', 'temperatureMinTime1', 'temperatureMax1', 'temperatureMaxTime1', 'apparentTemperatureMin1', 'apparentTemperatureMinTime1', 'apparentTemperatureMax1', 'apparentTemperatureMaxTime1', 'time1', 'sunriseTime2', 'sunsetTime2', 'moonPhase2', 'precipIntensity2', 'precipIntensityMax2', 'precipProbability2', 'temperatureHigh2', 'temperatureHighTime2', 'temperatureLow2', 'temperatureLowTime2', 'apparentTemperatureHigh2', 'apparentTemperatureHig

In [ ]:
sentinel_frame = pd.read_csv('/content/drive/My Drive/BerkeleyResults/Approach2/original_mosquito_weather_semi_clean.csv')

sentinel_frame["TRAPSET"]= pd.to_datetime(sentinel_frame["TRAPSET"])
sentinel_frame["TRAPCOLLECT"]= pd.to_datetime(sentinel_frame["TRAPCOLLECT"])
sentinel_frame["DIFF_DAYS"] = (sentinel_frame["TRAPCOLLECT"] - sentinel_frame["TRAPSET"]).dt.days
sentinel_frame["TRAPSET"]= sentinel_frame["TRAPSET"].dt.week
sentinel_frame["TRAPCOLLECT"]= sentinel_frame["TRAPCOLLECT"].dt.week

for category in categorical_columns:
    sentinel_frame[category] = pd.Categorical(sentinel_frame[category].astype('category')).codes

categorical_column_sizes = [len(sentinel_frame[column].cat.categories) for column in categorical_columns]
categorical_embedding_sizes = [(col_size, min(50, (col_size+1)//2)) for col_size in categorical_column_sizes]

for col in numerical_columns:
    sentinel_frame[col] = (sentinel_frame[col] - sentinel_frame[col].mean()) / (sentinel_frame[col].max() - sentinel_frame[col].min())
    
def get_categorical_tensor(in_frame, categories):
    categorical_data = np.stack([in_frame[col].cat.codes.values for col in categories], 1)
    categorical_data = torch.tensor(categorical_data, dtype=torch.int64)

    return categorical_data

def get_numerical_tensor(in_frame, numerical_columns):
    numerical_data = np.stack([in_frame[col].values for col in numerical_columns], 1)
    numerical_data = torch.tensor(numerical_data, dtype=torch.float)
    return numerical_data

In [ ]:
cor = sentinel_frame.corr(method='kendall')

x = cor["TOTAL"].sort_values(ascending=False)
pd.set_option('display.max_rows', len(x))

print(x)

TOTAL                            1.000000
GENUS                            0.248271
DIFF_DAYS                        0.149163
SPECIES                          0.121674
SETTIMEOFDAY                     0.119153
uvIndex13                        0.089484
uvIndex5                         0.085399
uvIndex12                        0.084473
TRAPTYPE                         0.083162
OBJECTID                         0.076821
uvIndex6                         0.062301
uvIndex9                         0.061502
windBearing4                     0.059807
COLLECTTIMEOFDAY                 0.056405
uvIndex11                        0.055301
windBearing5                     0.053272
uvIndex14                        0.053087
uvIndex4                         0.051399
temperatureHigh13                0.050853
temperatureMax13                 0.050343
windBearing13                    0.049621
uvIndex10                        0.048761
uvIndex2                         0.043029
apparentTemperatureHigh13        0

In [ ]:
cor = sentinel_frame.corr(method='spearman')

x = cor["TOTAL"].sort_values(ascending=False)
pd.set_option('display.max_rows', len(x))

print(x)

TOTAL                            1.000000
GENUS                            0.295357
DIFF_DAYS                        0.176961
SPECIES                          0.157594
SETTIMEOFDAY                     0.141359
uvIndex13                        0.121731
uvIndex5                         0.114633
uvIndex12                        0.114129
OBJECTID                         0.112038
TRAPTYPE                         0.103168
windBearing4                     0.087695
uvIndex6                         0.083453
uvIndex9                         0.083335
windBearing5                     0.078345
uvIndex11                        0.075434
temperatureHigh13                0.074827
temperatureMax13                 0.074152
windBearing13                    0.073341
uvIndex14                        0.071498
uvIndex4                         0.069487
COLLECTTIMEOFDAY                 0.066916
uvIndex10                        0.065802
apparentTemperatureHigh13        0.059992
cloudCover8                      0

In [ ]:
cor = sentinel_frame.corr(method='pearson')

x = cor["TOTAL"].sort_values(ascending=False)
pd.set_option('display.max_rows', len(x))

print(x)

TOTAL                            1.000000
GENUS                            0.195053
uvIndex12                        0.171735
uvIndex5                         0.132725
uvIndex6                         0.128913
uvIndex10                        0.113827
OBJECTID                         0.111428
uvIndex14                        0.109167
uvIndex13                        0.107372
uvIndex11                        0.105819
TRAPTYPE                         0.099526
temperatureMax12                 0.099281
temperatureHigh12                0.098960
temperatureHigh13                0.098953
temperatureMax13                 0.098278
precipIntensity13                0.093832
apparentTemperatureHigh13        0.092073
uvIndex4                         0.091778
uvIndex9                         0.090533
apparentTemperatureMax13         0.089983
temperatureLow12                 0.087704
uvIndex2                         0.086321
DIFF_DAYS                        0.086045
windBearing14                    0

In [ ]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_tensors[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))
    
class SwishM(nn.Module):
    def forward(self, input_tensor):
        return Swish.apply(input_tensor)
        
def create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout):
    all_layers = []
    num_categorical_cols = sum((nf for ni, nf in embedding_size))
    input_size = num_categorical_cols + num_numerical_cols
    print(input_size)
    print(num_categorical_cols)
    print(num_numerical_cols)

    all_layers.append(nn.Linear(input_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, output_size))
    all_layers.append(nn.ReLU())

    return nn.Sequential(*all_layers)

class Sentinel_net(nn.Module):

    def __init__(self, embedding_size, num_numerical_cols, output_size, dropout, hidden_size):
        super(Sentinel_net, self).__init__()
        self.layers = create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout)
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(dropout)
        self.batch_norm_num = nn.BatchNorm1d(num_numerical_cols)

    def forward(self, x_categorical, x_numerical):

        embeddings = []
        for i, e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:, i]))
        y = torch.cat(embeddings, 1)
        y = self.embedding_dropout(y)
        x_numerical = self.batch_norm_num(x_numerical)
        y = torch.cat([y, x_numerical], 1)
        y = self.layers(y)

        return y

In [ ]:
def train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr):
    model = Sentinel_net(categorical_embedding_sizes, train_n_t.shape[1], 1, dropout, hidden_size)
    model.train().cuda()

    criterion = torch.nn.MSELoss(reduction='mean') 
    params_dict = dict(model.named_parameters())
    params = []
    best = sys.maxsize
    r_value = 0
    std_err = 0
    
    for key, value in params_dict.items():
        if 'weight' in key and 'layers' in key:
            params += [{'params':value, 'weight_decay':4e-3}]
        else:
            params += [{'params':value, 'weight_decay':0.0}]

    optimizer = torch.optim.RMSprop(params, lr=lr, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0.9, centered=False)
    epoch = 0

    for i in range(1, 1000):
        y_pred = model(train_c_t, train_n_t)
        loss = criterion(y_pred.squeeze(), outputs_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            y_pred2 = model(test_c_t, test_n_t)
            c_out, c_pred = outputs_test.cpu().detach().numpy(), y_pred2.cpu().detach().numpy().squeeze()
            current_score = mean_absolute_error(c_out, c_pred)
            if current_score < best:
                best = current_score
                m, b, r_value, p_value, std_err = stats.linregress(c_out,c_pred)

            model.train()
    
    #print('MAE = ',current_score, 'r2 = ',r_value, 'SE = ',std_err)
    return best, r_value, std_err

In [ ]:
start_time = time.time()
learning = [0.01, 0.001, 0.0001] 
best_score_k_fold = sys.maxsize

print('2 Layers Architecture')

for dropout in range(0,7):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]

                train_c_t = get_categorical_tensor(train, categorical_columns).cuda()
                test_c_t = get_categorical_tensor(test, categorical_columns).cuda()
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
learning = [0.01, 0.001, 0.0001] 
best_score_k_fold = sys.maxsize

print('2 Layers Architecture')

for dropout in range(0,7):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]

                train_c_t = get_categorical_tensor(train, categorical_columns).cuda()
                test_c_t = get_categorical_tensor(test, categorical_columns).cuda()
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

2 Layers Architecture
MAE =  10.44150276184082 r2 =  0.4735204603512761 SE =  0.018962725548529825 lr =  0.01 hz =  50 dr =  0.0
Best so far 10.44150276184082

MAE =  10.416043663024903 r2 =  0.4425709201315044 SE =  0.03063144180526871 lr =  0.001 hz =  50 dr =  0.0
Best so far 10.416043663024903

MAE =  12.36879596710205 r2 =  0.38701680080382755 SE =  0.018612454872564344 lr =  0.0001 hz =  50 dr =  0.0
MAE =  10.384784698486328 r2 =  0.5742010875379008 SE =  0.021844672307338643 lr =  0.01 hz =  60 dr =  0.0
Best so far 10.384784698486328

MAE =  10.215986633300782 r2 =  0.4946960813341617 SE =  0.019116815450524585 lr =  0.001 hz =  60 dr =  0.0
Best so far 10.215986633300782

MAE =  12.230729675292968 r2 =  0.42538641617659306 SE =  0.017117749332677592 lr =  0.0001 hz =  60 dr =  0.0
MAE =  10.413243293762207 r2 =  0.5088247255310575 SE =  0.023647340982468963 lr =  0.01 hz =  70 dr =  0.0
MAE =  10.65552577972412 r2 =  0.42515144694248913 SE =  0.0215234444084986 lr =  0.001 hz

In [ ]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_tensors[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))
    
class SwishM(nn.Module):
    def forward(self, input_tensor):
        return Swish.apply(input_tensor)
        
def create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout):
    all_layers = []
    num_categorical_cols = sum((nf for ni, nf in embedding_size))
    input_size = num_categorical_cols + num_numerical_cols
    all_layers.append(nn.Linear(input_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, output_size))
    all_layers.append(nn.ReLU())

    return nn.Sequential(*all_layers)

class Sentinel_net(nn.Module):

    def __init__(self, embedding_size, num_numerical_cols, output_size, dropout, hidden_size):
        super(Sentinel_net, self).__init__()
        self.layers = create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout)
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(dropout)
        self.batch_norm_num = nn.BatchNorm1d(num_numerical_cols)

    def forward(self, x_categorical, x_numerical):

        embeddings = []
        for i, e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:, i]))
        y = torch.cat(embeddings, 1)
        y = self.embedding_dropout(y)
        x_numerical = self.batch_norm_num(x_numerical)
        y = torch.cat([y, x_numerical], 1)
        y = self.layers(y)

        return y

In [ ]:
start_time = time.time()
learning = [0.01, 0.001, 0.0001] 
best_score_k_fold = sys.maxsize

print('3 Layers Architecture')

for dropout in range(0,7):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]

                train_c_t = get_categorical_tensor(train, categorical_columns).cuda()
                test_c_t = get_categorical_tensor(test, categorical_columns).cuda()
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

3 Layers Architecture
MAE =  10.701027297973633 r2 =  0.47506872669979316 SE =  0.02469262572123821 lr =  0.01 hz =  50 dr =  0.0
Best so far 10.701027297973633

MAE =  10.71508560180664 r2 =  0.42670408406334726 SE =  0.028432827230967222 lr =  0.001 hz =  50 dr =  0.0
MAE =  12.471528244018554 r2 =  0.4207684491508464 SE =  0.025458205828203007 lr =  0.0001 hz =  50 dr =  0.0
MAE =  10.352318382263183 r2 =  0.3988593470582014 SE =  0.020142443449138674 lr =  0.01 hz =  60 dr =  0.0
Best so far 10.352318382263183

MAE =  11.008835983276366 r2 =  0.4086742528145016 SE =  0.030355636394596174 lr =  0.001 hz =  60 dr =  0.0
MAE =  12.387226486206055 r2 =  0.4754725914890125 SE =  0.024954498055770522 lr =  0.0001 hz =  60 dr =  0.0
MAE =  10.030201721191407 r2 =  0.5333836107031337 SE =  0.019751844920933726 lr =  0.01 hz =  70 dr =  0.0
Best so far 10.030201721191407

MAE =  10.60122184753418 r2 =  0.47807338089623724 SE =  0.03182642755952204 lr =  0.001 hz =  70 dr =  0.0
MAE =  12.40

In [ ]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_tensors[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))
    
class SwishM(nn.Module):
    def forward(self, input_tensor):
        return Swish.apply(input_tensor)
        
def create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout):
    all_layers = []
    num_categorical_cols = sum((nf for ni, nf in embedding_size))
    input_size = num_categorical_cols + num_numerical_cols
    all_layers.append(nn.Linear(input_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))
    
    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, output_size))
    all_layers.append(nn.ReLU())

    return nn.Sequential(*all_layers)

class Sentinel_net(nn.Module):

    def __init__(self, embedding_size, num_numerical_cols, output_size, dropout, hidden_size):
        super(Sentinel_net, self).__init__()
        self.layers = create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout)
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(dropout)
        self.batch_norm_num = nn.BatchNorm1d(num_numerical_cols)

    def forward(self, x_categorical, x_numerical):

        embeddings = []
        for i, e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:, i]))
        y = torch.cat(embeddings, 1)
        y = self.embedding_dropout(y)
        x_numerical = self.batch_norm_num(x_numerical)
        y = torch.cat([y, x_numerical], 1)
        y = self.layers(y)

        return y

In [ ]:
start_time = time.time()
learning = [0.01, 0.001, 0.0001] 
best_score_k_fold = sys.maxsize

print('4 Layers Architecture')

for dropout in range(0,7):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]

                train_c_t = get_categorical_tensor(train, categorical_columns).cuda()
                test_c_t = get_categorical_tensor(test, categorical_columns).cuda()
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

4 Layers Architecture
MAE =  10.17833595275879 r2 =  0.5105020952024393 SE =  0.024959930171531843 lr =  0.01 hz =  50 dr =  0.0
Best so far 10.17833595275879

MAE =  10.837935829162598 r2 =  0.428826877117597 SE =  0.02878678992704823 lr =  0.001 hz =  50 dr =  0.0
MAE =  12.060610008239745 r2 =  0.3792604726658298 SE =  0.02592526099457343 lr =  0.0001 hz =  50 dr =  0.0
MAE =  10.439393424987793 r2 =  0.4470146227661774 SE =  0.026199307267228543 lr =  0.01 hz =  60 dr =  0.0
MAE =  10.92205867767334 r2 =  0.39677453297495713 SE =  0.03147178034579588 lr =  0.001 hz =  60 dr =  0.0
MAE =  12.162130546569824 r2 =  0.4694594649440802 SE =  0.02800022178257166 lr =  0.0001 hz =  60 dr =  0.0
MAE =  9.961118698120117 r2 =  0.5317800284705706 SE =  0.022017595081679126 lr =  0.01 hz =  70 dr =  0.0
Best so far 9.961118698120117

MAE =  10.661465454101563 r2 =  0.41434630309612197 SE =  0.030771746962623896 lr =  0.001 hz =  70 dr =  0.0
MAE =  12.087113189697266 r2 =  0.3421117463402183 

In [ ]:
categorical_columns = ['summary14']
numerical_columns = ['temperatureLowTime5', 'apparentTemperatureHighTime5', 'apparentTemperatureLow5', 'apparentTemperatureLowTime5']

In [ ]:
sentinel_frame = pd.read_csv('/content/drive/My Drive/Colab/Mosquito_Berkeley/Approach2/original_mosquito_weather_semi_clean.csv')

sentinel_frame["TRAPSET"]= pd.to_datetime(sentinel_frame["TRAPSET"])
sentinel_frame["TRAPCOLLECT"]= pd.to_datetime(sentinel_frame["TRAPCOLLECT"])
sentinel_frame["DIFF_DAYS"] = (sentinel_frame["TRAPCOLLECT"] - sentinel_frame["TRAPSET"]).dt.days
sentinel_frame["TRAPSET"]= sentinel_frame["TRAPSET"].dt.week
sentinel_frame["TRAPCOLLECT"]= sentinel_frame["TRAPCOLLECT"].dt.week

for category in categorical_columns:
    sentinel_frame[category] = sentinel_frame[category].astype('category')

categorical_column_sizes = [len(sentinel_frame[column].cat.categories) for column in categorical_columns]
categorical_embedding_sizes = [(col_size, min(50, (col_size+1)//2)) for col_size in categorical_column_sizes]

for col in numerical_columns:
    sentinel_frame[col] = (sentinel_frame[col] - sentinel_frame[col].mean()) / (sentinel_frame[col].max() - sentinel_frame[col].min())
    
def get_categorical_tensor(in_frame, categories):
    categorical_data = np.stack([in_frame[col].cat.codes.values for col in categories], 1)
    categorical_data = torch.tensor(categorical_data, dtype=torch.int64)

    return categorical_data

def get_numerical_tensor(in_frame, numerical_columns):
    numerical_data = np.stack([in_frame[col].values for col in numerical_columns], 1)
    numerical_data = torch.tensor(numerical_data, dtype=torch.float)
    return numerical_data

In [ ]:
start_time = time.time()
learning = [0.01, 0.001, 0.0001] 
best_score_k_fold = sys.maxsize

print('4 Layers Architecture')

for dropout in range(2,7):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(100, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]

                train_c_t = get_categorical_tensor(train, categorical_columns).cuda()
                test_c_t = get_categorical_tensor(test, categorical_columns).cuda()
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

4 Layers Architecture
MAE =  11.028680229187012 r2 =  0.21562661246087575 SE =  0.010164354974308428 lr =  0.01 hz =  100 dr =  0.2
Best so far 11.028680229187012

MAE =  11.35018539428711 r2 =  0.2702006992768822 SE =  0.013546306202781622 lr =  0.001 hz =  100 dr =  0.2
MAE =  12.199255180358886 r2 =  0.21956835448068776 SE =  0.013955451656640841 lr =  0.0001 hz =  100 dr =  0.2
MAE =  10.89233627319336 r2 =  0.3224584131355987 SE =  0.011169318701322867 lr =  0.01 hz =  110 dr =  0.2
Best so far 10.89233627319336

MAE =  11.63238754272461 r2 =  0.25619678338340923 SE =  0.014674106583706015 lr =  0.001 hz =  110 dr =  0.2
MAE =  12.281262969970703 r2 =  0.20124844196291533 SE =  0.016373412977721772 lr =  0.0001 hz =  110 dr =  0.2
MAE =  10.88133487701416 r2 =  0.2599238816206778 SE =  0.011018583131389932 lr =  0.01 hz =  120 dr =  0.2
Best so far 10.88133487701416

MAE =  11.653334999084473 r2 =  0.2739311742012387 SE =  0.014857419896308798 lr =  0.001 hz =  120 dr =  0.2
MAE =

KeyboardInterrupt: ignored

In [ ]:
categorical_columns = ['summary14']
numerical_columns = ['uvIndex12', 'apparentTemperatureHighTime5', 'apparentTemperatureLow5', 'apparentTemperatureLowTime5',
                     'dewPoint5', 'humidity5', 'apparentTemperatureHigh5', 'temperatureLow5', 'sunriseTime5']

In [ ]:
sentinel_frame = pd.read_csv('/content/drive/My Drive/Colab/Mosquito_Berkeley/Approach2/original_mosquito_weather_semi_clean.csv')

sentinel_frame["TRAPSET"]= pd.to_datetime(sentinel_frame["TRAPSET"])
sentinel_frame["TRAPCOLLECT"]= pd.to_datetime(sentinel_frame["TRAPCOLLECT"])
sentinel_frame["DIFF_DAYS"] = (sentinel_frame["TRAPCOLLECT"] - sentinel_frame["TRAPSET"]).dt.days
sentinel_frame["TRAPSET"]= sentinel_frame["TRAPSET"].dt.week
sentinel_frame["TRAPCOLLECT"]= sentinel_frame["TRAPCOLLECT"].dt.week

for category in categorical_columns:
    sentinel_frame[category] = sentinel_frame[category].astype('category')

categorical_column_sizes = [len(sentinel_frame[column].cat.categories) for column in categorical_columns]
categorical_embedding_sizes = [(col_size, min(50, (col_size+1)//2)) for col_size in categorical_column_sizes]

for col in numerical_columns:
    sentinel_frame[col] = (sentinel_frame[col] - sentinel_frame[col].mean()) / (sentinel_frame[col].max() - sentinel_frame[col].min())
    
def get_categorical_tensor(in_frame, categories):
    categorical_data = np.stack([in_frame[col].cat.codes.values for col in categories], 1)
    categorical_data = torch.tensor(categorical_data, dtype=torch.int64)

    return categorical_data

def get_numerical_tensor(in_frame, numerical_columns):
    numerical_data = np.stack([in_frame[col].values for col in numerical_columns], 1)
    numerical_data = torch.tensor(numerical_data, dtype=torch.float)
    return numerical_data

In [ ]:
start_time = time.time()
learning = [0.01, 0.001, 0.0001] 
best_score_k_fold = sys.maxsize

print('4 Layers Architecture')

for dropout in range(2,7):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(100, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]

                train_c_t = get_categorical_tensor(train, categorical_columns).cuda()
                test_c_t = get_categorical_tensor(test, categorical_columns).cuda()
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

4 Layers Architecture
MAE =  10.889762496948242 r2 =  0.22155856300788584 SE =  0.009470752298873304 lr =  0.01 hz =  100 dr =  0.2
Best so far 10.889762496948242

MAE =  11.643433380126954 r2 =  0.2376772283501239 SE =  0.015082590961337577 lr =  0.001 hz =  100 dr =  0.2
MAE =  12.25010643005371 r2 =  0.1500314149434342 SE =  0.009545107523240049 lr =  0.0001 hz =  100 dr =  0.2
MAE =  10.913806915283203 r2 =  0.22918643360185958 SE =  0.010996980069259634 lr =  0.01 hz =  110 dr =  0.2
MAE =  11.77350730895996 r2 =  0.20931227933354074 SE =  0.013966705706160732 lr =  0.001 hz =  110 dr =  0.2
MAE =  12.131224632263184 r2 =  0.13713704899011547 SE =  0.008468274646365594 lr =  0.0001 hz =  110 dr =  0.2
MAE =  10.843844604492187 r2 =  0.2541602508993093 SE =  0.011532782610469255 lr =  0.01 hz =  120 dr =  0.2
Best so far 10.843844604492187

MAE =  11.672551918029786 r2 =  0.24509162964889414 SE =  0.017060663582018385 lr =  0.001 hz =  120 dr =  0.2
MAE =  12.218270683288575 r2 =  

In [ ]:
categorical_columns = ['TRAPTYPE', 'ATTRACTANTSUSED', 'SETTIMEOFDAY', 'COLLECTTIMEOFDAY', 'GENUS', 'SPECIES', 'TRAPID','TRAPSITE']
numerical_columns = ['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR','TRAPCOLLECT','TRAPSET','TRAPCOLLECT', 'DIFF_DAYS','TOTAL']

weather_columns = ['sunriseTime', 'sunsetTime', 'moonPhase', 
                     'precipIntensity', 'precipIntensityMax', 'precipProbability', 'temperatureHigh', 'temperatureHighTime',
                     'temperatureLow', 'temperatureLowTime', 'apparentTemperatureHigh', 'apparentTemperatureHighTime',
                     'apparentTemperatureLow', 'apparentTemperatureLowTime', 'dewPoint', 'humidity', 'pressure', 'windSpeed', 'windGust', 
                     'windGustTime', 'windBearing', 'cloudCover', 'uvIndex', 'uvIndexTime','visibility', 'temperatureMin',
                     'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime', 'apparentTemperatureMin','apparentTemperatureMinTime', 'apparentTemperatureMax',
                     'apparentTemperatureMaxTime','time']

for i in range(1,15):
    for item in weather_columns:
        numerical_columns.append(item + str(i))

print(numerical_columns)

['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR', 'TRAPCOLLECT', 'TRAPSET', 'TRAPCOLLECT', 'DIFF_DAYS', 'TOTAL', 'sunriseTime1', 'sunsetTime1', 'moonPhase1', 'precipIntensity1', 'precipIntensityMax1', 'precipProbability1', 'temperatureHigh1', 'temperatureHighTime1', 'temperatureLow1', 'temperatureLowTime1', 'apparentTemperatureHigh1', 'apparentTemperatureHighTime1', 'apparentTemperatureLow1', 'apparentTemperatureLowTime1', 'dewPoint1', 'humidity1', 'pressure1', 'windSpeed1', 'windGust1', 'windGustTime1', 'windBearing1', 'cloudCover1', 'uvIndex1', 'uvIndexTime1', 'visibility1', 'temperatureMin1', 'temperatureMinTime1', 'temperatureMax1', 'temperatureMaxTime1', 'apparentTemperatureMin1', 'apparentTemperatureMinTime1', 'apparentTemperatureMax1', 'apparentTemperatureMaxTime1', 'time1', 'sunriseTime2', 'sunsetTime2', 'moonPhase2', 'precipIntensity2', 'precipIntensityMax2', 'precipProbability2', 'temperatureHigh2', 'temperatureHighTime2', 'temperatureLow2', 'temperatureLowTime2', 'apparentTempera

In [ ]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

sentinel_frame = pd.read_csv('/content/drive/My Drive/Colab/Mosquito_Berkeley/Approach2/original_mosquito_weather_semi_clean.csv')
sentinel_frame["TRAPSET"]= pd.to_datetime(sentinel_frame["TRAPSET"])
sentinel_frame["TRAPCOLLECT"]= pd.to_datetime(sentinel_frame["TRAPCOLLECT"])
sentinel_frame["DIFF_DAYS"] = (sentinel_frame["TRAPCOLLECT"] - sentinel_frame["TRAPSET"]).dt.days
sentinel_frame["TRAPSET"]= sentinel_frame["TRAPSET"].dt.week
sentinel_frame["TRAPCOLLECT"]= sentinel_frame["TRAPCOLLECT"].dt.week

for category in categorical_columns:
    sentinel_frame[category] = sentinel_frame[category].astype('category')

categorical_column_sizes = [len(sentinel_frame[column].cat.categories) for column in categorical_columns]
categorical_embedding_sizes = [(col_size, min(50, (col_size+1)//2)) for col_size in categorical_column_sizes]

#Using Pearson Correlation
cor = sentinel_frame.corr(method='spearman')


for col in numerical_columns:
    sentinel_frame[col] = (sentinel_frame[col] - sentinel_frame[col].mean()) / (sentinel_frame[col].max() - sentinel_frame[col].min())
    
def get_categorical_tensor(in_frame, categories):
    categorical_data = np.stack([in_frame[col].cat.codes.values for col in categories], 1)
    categorical_data = torch.tensor(categorical_data, dtype=torch.int64)

    return categorical_data

def get_numerical_tensor(in_frame, numerical_columns):
    numerical_data = np.stack([in_frame[col].values for col in numerical_columns], 1)
    numerical_data = torch.tensor(numerical_data, dtype=torch.float)
    return numerical_data

In [ ]:
cor = sentinel_frame.corr(method='spearman')

x = cor["TOTAL"].sort_values(ascending=False)
pd.set_option('display.max_rows', len(x))

print(x)

TOTAL                            1.000000
uvIndex12                        0.171735
uvIndex5                         0.132725
uvIndex6                         0.128913
uvIndex10                        0.113827
uvIndex14                        0.109167
uvIndex13                        0.107372
uvIndex11                        0.105819
temperatureMax12                 0.099281
temperatureHigh12                0.098960
temperatureHigh13                0.098953
temperatureMax13                 0.098278
precipIntensity13                0.093832
apparentTemperatureHigh13        0.092073
uvIndex4                         0.091778
uvIndex9                         0.090533
apparentTemperatureMax13         0.089983
temperatureLow12                 0.087704
uvIndex2                         0.086321
DIFF_DAYS                        0.086045
windBearing14                    0.085944
apparentTemperatureLow12         0.084777
temperatureMin13                 0.080017
apparentTemperatureMin13         0

In [ ]:
x = cor["TOTAL"].sort_values(ascending=False)
pd.set_option('display.max_rows', len(x))

print(x)

TOTAL                            1.000000
DIFF_DAYS                        0.149163
uvIndex13                        0.089484
uvIndex5                         0.085399
uvIndex12                        0.084473
OBJECTID                         0.076821
uvIndex6                         0.062301
uvIndex9                         0.061502
windBearing4                     0.059807
uvIndex11                        0.055301
windBearing5                     0.053272
uvIndex14                        0.053087
uvIndex4                         0.051399
temperatureHigh13                0.050853
temperatureMax13                 0.050343
windBearing13                    0.049621
uvIndex10                        0.048761
uvIndex2                         0.043029
apparentTemperatureHigh13        0.041406
pressure10                       0.041327
cloudCover8                      0.040797
apparentTemperatureMax13         0.039625
moonPhase14                      0.033368
windBearing6                     0

In [ ]:
x = cor["TOTAL"].sort_values(ascending=False)
pd.set_option('display.max_rows', len(x))

print(x)

TOTAL                            1.000000
DIFF_DAYS                        0.176961
uvIndex13                        0.121731
uvIndex5                         0.114633
uvIndex12                        0.114129
OBJECTID                         0.112038
windBearing4                     0.087695
uvIndex6                         0.083453
uvIndex9                         0.083335
windBearing5                     0.078345
uvIndex11                        0.075434
temperatureHigh13                0.074827
temperatureMax13                 0.074152
windBearing13                    0.073341
uvIndex14                        0.071498
uvIndex4                         0.069487
uvIndex10                        0.065802
apparentTemperatureHigh13        0.059992
cloudCover8                      0.059768
pressure10                       0.058898
uvIndex2                         0.058653
apparentTemperatureMax13         0.057539
moonPhase14                      0.047603
windBearing6                     0

In [ ]:
# categorical_columns = ['TRAPTYPE', 'ATTRACTANTSUSED', 'SETTIMEOFDAY', 'COLLECTTIMEOFDAY', 'GENUS', 'SPECIES', 'TRAPID','TRAPSITE']
# numerical_columns = ['precipIntensityMaxTime12', 'precipIntensityMaxTime14', 'apparentTemperatureMinTime13', 'sunsetTime4','sunsetTime6',
#                      'sunsetTime1','sunriseTime12','temperatureLowTime10', 'windGustTime13', 'sunsetTime3',
#                      'DIFF_DAYS', 'uvIndex13', 'uvIndex5', 'uvIndex12']

categorical_columns = ['TRAPTYPE', 'ATTRACTANTSUSED', 'SETTIMEOFDAY', 'COLLECTTIMEOFDAY', 'GENUS', 'SPECIES', 'TRAPID','TRAPSITE']
numerical_columns = ['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR','TRAPCOLLECT','TRAPSET','TRAPCOLLECT', 'DIFF_DAYS']

weather_columns = ['sunriseTime', 'sunsetTime', 'moonPhase', 
                     'precipIntensity', 'precipIntensityMax', 'precipProbability', 'temperatureHigh', 'temperatureHighTime',
                     'temperatureLow', 'temperatureLowTime', 'apparentTemperatureHigh', 'apparentTemperatureHighTime',
                     'apparentTemperatureLow', 'apparentTemperatureLowTime', 'dewPoint', 'humidity', 'pressure', 'windSpeed', 'windGust', 
                     'windGustTime', 'windBearing', 'cloudCover', 'uvIndex', 'uvIndexTime','visibility', 'temperatureMin',
                     'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime', 'apparentTemperatureMin','apparentTemperatureMinTime', 'apparentTemperatureMax',
                     'apparentTemperatureMaxTime','time']

for i in range(1,2):
    for item in weather_columns:
        numerical_columns.append(item + str(14))

print(numerical_columns)

['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR', 'TRAPCOLLECT', 'TRAPSET', 'TRAPCOLLECT', 'DIFF_DAYS', 'sunriseTime14', 'sunsetTime14', 'moonPhase14', 'precipIntensity14', 'precipIntensityMax14', 'precipProbability14', 'temperatureHigh14', 'temperatureHighTime14', 'temperatureLow14', 'temperatureLowTime14', 'apparentTemperatureHigh14', 'apparentTemperatureHighTime14', 'apparentTemperatureLow14', 'apparentTemperatureLowTime14', 'dewPoint14', 'humidity14', 'pressure14', 'windSpeed14', 'windGust14', 'windGustTime14', 'windBearing14', 'cloudCover14', 'uvIndex14', 'uvIndexTime14', 'visibility14', 'temperatureMin14', 'temperatureMinTime14', 'temperatureMax14', 'temperatureMaxTime14', 'apparentTemperatureMin14', 'apparentTemperatureMinTime14', 'apparentTemperatureMax14', 'apparentTemperatureMaxTime14', 'time14']


In [ ]:
sentinel_frame = pd.read_csv('/content/drive/My Drive/Colab/Mosquito_Berkeley/Approach2/original_mosquito_weather_semi_clean.csv')

sentinel_frame["TRAPSET"]= pd.to_datetime(sentinel_frame["TRAPSET"])
sentinel_frame["TRAPCOLLECT"]= pd.to_datetime(sentinel_frame["TRAPCOLLECT"])
sentinel_frame["DIFF_DAYS"] = (sentinel_frame["TRAPCOLLECT"] - sentinel_frame["TRAPSET"]).dt.days
sentinel_frame["TRAPSET"]= sentinel_frame["TRAPSET"].dt.week
sentinel_frame["TRAPCOLLECT"]= sentinel_frame["TRAPCOLLECT"].dt.week

for category in categorical_columns:
    sentinel_frame[category] = sentinel_frame[category].astype('category')

categorical_column_sizes = [len(sentinel_frame[column].cat.categories) for column in categorical_columns]
categorical_embedding_sizes = [(col_size, min(50, (col_size+1)//2)) for col_size in categorical_column_sizes]

for col in numerical_columns:
    sentinel_frame[col] = (sentinel_frame[col] - sentinel_frame[col].mean()) / (sentinel_frame[col].max() - sentinel_frame[col].min())
    
def get_categorical_tensor(in_frame, categories):
    categorical_data = np.stack([in_frame[col].cat.codes.values for col in categories], 1)
    categorical_data = torch.tensor(categorical_data, dtype=torch.int64)

    return categorical_data

def get_numerical_tensor(in_frame, numerical_columns):
    numerical_data = np.stack([in_frame[col].values for col in numerical_columns], 1)
    numerical_data = torch.tensor(numerical_data, dtype=torch.float)
    return numerical_data

In [ ]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_tensors[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))
    
class SwishM(nn.Module):
    def forward(self, input_tensor):
        return Swish.apply(input_tensor)
        
def create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout):
    all_layers = []
    num_categorical_cols = sum((nf for ni, nf in embedding_size))
    input_size = num_categorical_cols + num_numerical_cols
    all_layers.append(nn.Linear(input_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, output_size))
    all_layers.append(nn.ReLU())

    return nn.Sequential(*all_layers)

class Sentinel_net(nn.Module):

    def __init__(self, embedding_size, num_numerical_cols, output_size, dropout, hidden_size):
        super(Sentinel_net, self).__init__()
        self.layers = create_neural_modules(embedding_size, num_numerical_cols, output_size, hidden_size, dropout)
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        self.embedding_dropout = nn.Dropout(dropout)
        self.batch_norm_num = nn.BatchNorm1d(num_numerical_cols)

    def forward(self, x_categorical, x_numerical):

        embeddings = []
        for i, e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:, i]))
        y = torch.cat(embeddings, 1)
        y = self.embedding_dropout(y)
        x_numerical = self.batch_norm_num(x_numerical)
        y = torch.cat([y, x_numerical], 1)
        y = self.layers(y)

        return y

In [ ]:
def train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr):
    model = Sentinel_net(categorical_embedding_sizes, train_n_t.shape[1], 1, dropout, hidden_size)
    model.train().cuda()

    criterion = torch.nn.MSELoss(reduction='mean') 
    params_dict = dict(model.named_parameters())
    params = []
    best = sys.maxsize
    r_value = 0
    std_err = 0
    
    for key, value in params_dict.items():
        if 'weight' in key and 'layers' in key:
            params += [{'params':value, 'weight_decay':4e-3}]
        else:
            params += [{'params':value, 'weight_decay':0.0}]

    optimizer = torch.optim.RMSprop(params, lr=lr, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0.9, centered=False)
    epoch = 0

    for i in range(1, 1000):
        y_pred = model(train_c_t, train_n_t)
        loss = criterion(y_pred.squeeze(), outputs_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            y_pred2 = model(test_c_t, test_n_t)
            c_out, c_pred = outputs_test.cpu().detach().numpy(), y_pred2.cpu().detach().numpy().squeeze()
            current_score = mean_absolute_error(c_out, c_pred)
            if current_score < best:
                best = current_score
                m, b, r_value, p_value, std_err = stats.linregress(c_out,c_pred)

            model.train()
    
    #print('MAE = ',current_score, 'r2 = ',r_value, 'SE = ',std_err)
    return best, r_value, std_err

In [ ]:
start_time = time.time()
learning = [0.01, 0.001, 0.0001] 
best_score_k_fold = sys.maxsize

print('4 Layers Architecture')

for dropout in range(2,7):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(100, 150, 10):
        for lr in learning:  

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]

                train_c_t = get_categorical_tensor(train, categorical_columns).cuda()
                test_c_t = get_categorical_tensor(test, categorical_columns).cuda()
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

4 Layers Architecture
MAE =  9.472059059143067 r2 =  0.6055633578544912 SE =  0.020846954265486932 lr =  0.01 hz =  100 dr =  0.2
Best so far 9.472059059143067

MAE =  9.773980522155762 r2 =  0.5457282745346899 SE =  0.02325936933029145 lr =  0.001 hz =  100 dr =  0.2
MAE =  10.244432830810547 r2 =  0.5365695604866858 SE =  0.029267228522954276 lr =  0.0001 hz =  100 dr =  0.2
MAE =  9.501375865936279 r2 =  0.5495815833210675 SE =  0.019294976853231397 lr =  0.01 hz =  110 dr =  0.2
MAE =  9.848257637023925 r2 =  0.5565727672384749 SE =  0.024050937776143727 lr =  0.001 hz =  110 dr =  0.2
MAE =  10.285157012939454 r2 =  0.5736361240515854 SE =  0.02757163542281809 lr =  0.0001 hz =  110 dr =  0.2
MAE =  9.69464406967163 r2 =  0.5898979147530419 SE =  0.020573062711355217 lr =  0.01 hz =  120 dr =  0.2
MAE =  9.74937744140625 r2 =  0.5882846893917788 SE =  0.023738010381260295 lr =  0.001 hz =  120 dr =  0.2
MAE =  10.39681282043457 r2 =  0.5350459187459837 SE =  0.027569568211476565 l

In [ ]:
start_time = time.time()
learning = [0.01, 0.001, 0.0001] 
best_score_k_fold = sys.maxsize

print('4 Layers Architecture')

for dropout in range(2,7):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(100, 150, 10):
        for lr in learning:  

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]

                train_c_t = get_categorical_tensor(train, categorical_columns).cuda()
                test_c_t = get_categorical_tensor(test, categorical_columns).cuda()
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_c_t,test_c_t,train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

4 Layers Architecture


TypeError: ignored

In [ ]:
numerical_columns = ['pressure13', 'humidity6', 'humidity12', 'humidity14',
                     'humidity11', 'pressure14','uvIndex12', 'uvIndex5',
                     'uvIndex6', 'uvIndex10', 'uvIndex14', 'uvIndex13','uvIndex11','TRAPSET','TRAPCOLLECT']

In [ ]:
sentinel_frame = pd.read_csv('/content/drive/My Drive/Colab/Mosquito_Berkeley/Approach2/original_mosquito_weather_semi_clean.csv')

sentinel_frame["TRAPSET"]= pd.to_datetime(sentinel_frame["TRAPSET"])
sentinel_frame["TRAPCOLLECT"]= pd.to_datetime(sentinel_frame["TRAPCOLLECT"])
sentinel_frame["DIFF_DAYS"] = (sentinel_frame["TRAPCOLLECT"] - sentinel_frame["TRAPSET"]).dt.days
sentinel_frame["TRAPSET"]= sentinel_frame["TRAPSET"].dt.week
sentinel_frame["TRAPCOLLECT"]= sentinel_frame["TRAPCOLLECT"].dt.week

sentinel_frame = sentinel_frame[['pressure13', 'humidity6', 'humidity12', 'humidity14',
                     'humidity11', 'pressure14','uvIndex12', 'uvIndex5',
                     'uvIndex6', 'uvIndex10', 'uvIndex14', 'uvIndex13','uvIndex11','TRAPSET','TRAPCOLLECT',
                     'temperatureMax12', 'temperatureHigh12', 'temperatureHigh13', 'temperatureMax13','precipIntensity13','apparentTemperatureHigh13','TRAPCOLLECT','TOTAL']]

for col in numerical_columns:
    sentinel_frame[col] = (sentinel_frame[col] - sentinel_frame[col].mean()) / (sentinel_frame[col].max() - sentinel_frame[col].min())
    
def get_categorical_tensor(in_frame, categories):
    categorical_data = np.stack([in_frame[col].cat.codes.values for col in categories], 1)
    categorical_data = torch.tensor(categorical_data, dtype=torch.int64)

    return categorical_data

def get_numerical_tensor(in_frame, numerical_columns):
    numerical_data = np.stack([in_frame[col].values for col in numerical_columns], 1)
    numerical_data = torch.tensor(numerical_data, dtype=torch.float)
    return numerical_data

In [ ]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

        
def create_neural_modules(num_numerical_cols, output_size, hidden_size, dropout):
    all_layers = []
    input_size = num_numerical_cols 
    all_layers.append(nn.Linear(input_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))
    
    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, output_size))
    all_layers.append(nn.ReLU())

    return nn.Sequential(*all_layers)

class Sentinel_net(nn.Module):

    def __init__(self, num_numerical_cols, output_size, dropout, hidden_size):
        super(Sentinel_net, self).__init__()
        self.layers = create_neural_modules(num_numerical_cols, output_size, hidden_size, dropout)
        self.batch_norm_num = nn.BatchNorm1d(num_numerical_cols)

    def forward(self, x_numerical):

        x_numerical = self.batch_norm_num(x_numerical)
        x_numerical = self.layers(x_numerical)

        return x_numerical

In [ ]:
def train_shallow_net(train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr):
    model = Sentinel_net(train_n_t.shape[1], 1, dropout, hidden_size)
    model.train().cuda()

    criterion = torch.nn.MSELoss(reduction='mean') 
    params_dict = dict(model.named_parameters())
    params = []
    best = sys.maxsize
    r_value = 0
    std_err = 0
    
    for key, value in params_dict.items():
        if 'weight' in key and 'layers' in key:
            params += [{'params':value, 'weight_decay':4e-3}]
        else:
            params += [{'params':value, 'weight_decay':0.0}]

    optimizer = torch.optim.RMSprop(params, lr=lr, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0.9, centered=False)
    epoch = 0

    for i in range(1, 1000):
        y_pred = model( train_n_t)
        loss = criterion(y_pred.squeeze(), outputs_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            y_pred2 = model( test_n_t)
            c_out, c_pred = outputs_test.cpu().detach().numpy(), y_pred2.cpu().detach().numpy().squeeze()
            current_score = mean_absolute_error(c_out, c_pred)
            if current_score < best:
                best = current_score
                m, b, r_value, p_value, std_err = stats.linregress(c_out,c_pred)

            model.train()
    
    #print('MAE = ',current_score, 'r2 = ',r_value, 'SE = ',std_err)
    return best, r_value, std_err

In [ ]:
start_time = time.time()
learning = [0.01, 0.001, 0.0001] 
best_score_k_fold = sys.maxsize

print('4 Layers Architecture')

for dropout in range(2,7):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(100, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame.iloc[train_index], sentinel_frame.iloc[test_index]
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

4 Layers Architecture
MAE =  10.888934707641601 r2 =  0.24507840790041638 SE =  0.009977577556411163 lr =  0.01 hz =  100 dr =  0.2
Best so far 10.888934707641601

MAE =  11.833579444885254 r2 =  0.2610217087362604 SE =  0.014741444660635367 lr =  0.001 hz =  100 dr =  0.2
MAE =  12.503955268859864 r2 =  0.22781815223868226 SE =  0.014935868506786612 lr =  0.0001 hz =  100 dr =  0.2
MAE =  10.843804359436035 r2 =  0.2633150841480563 SE =  0.008514258641656827 lr =  0.01 hz =  110 dr =  0.2
Best so far 10.843804359436035

MAE =  11.868379402160645 r2 =  0.25554351506406364 SE =  0.0149464558081716 lr =  0.001 hz =  110 dr =  0.2
MAE =  12.554525184631348 r2 =  0.23871550626974827 SE =  0.015456932270729742 lr =  0.0001 hz =  110 dr =  0.2
MAE =  10.851188278198242 r2 =  0.2600261950191808 SE =  0.008271527682696398 lr =  0.01 hz =  120 dr =  0.2
MAE =  11.696317672729492 r2 =  0.2624580877351213 SE =  0.014670535921641956 lr =  0.001 hz =  120 dr =  0.2
MAE =  12.478167533874512 r2 =  0